# Downtown Boston SUMO Network Setup and Testing

This notebook:
1. Sets up the Downtown Boston network
2. Tests the data collection
3. Visualizes some initial metrics

In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

## 1. Network Setup

First, let's set up our Downtown Boston network using the BostonNetworkSetup class.

In [2]:
from src.environment.boston_network_setup import BostonNetworkSetup

# Initialize and run setup
setup = BostonNetworkSetup(config={})
setup.setup_network()

--2024-10-24 17:07:32--  https://api.openstreetmap.org/api/0.6/map?bbox=42.348,-71.071,42.366,-71.051
Resolving api.openstreetmap.org (api.openstreetmap.org)... 104.21.88.66, 172.67.173.161, 2606:4700:3030::ac43:ada1, ...
Connecting to api.openstreetmap.org (api.openstreetmap.org)|104.21.88.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/xml]
Saving to: ‘data/sumo_nets/boston/downtown_boston.osm’

     0K                                                        3.15M=0s

2024-10-24 17:07:32 (3.15 MB/s) - ‘data/sumo_nets/boston/downtown_boston.osm’ saved [387]



Generating SUMO network...
Generating traffic demand...
Creating configuration files...
Network setup complete!


Error: On processing option '--osm.stop-output-file':
 No option with the name 'osm.stop-output-file' exists.
Error: The parameter 'data/sumo_nets/boston/stops.add.xml' is not allowed in this context.
 Switch or parameter name expected.
Error: Could not parse commandline options.
Quitting (on error).
Error: Could not open net-file 'data/sumo_nets/boston/downtown_boston.net.xml'.
Quitting (on error).
python: can't open file '/Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/notebooks/$SUMO_HOME/tools/randomTrips.py': [Errno 2] No such file or directory
python: can't open file '/Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/notebooks/$SUMO_HOME/tools/randomTrips.py': [Errno 2] No such file or directory
python: can't open file '/Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/notebooks/$SUMO_HOME/tools/randomTrips.py': [Errno 2] No such file or directory


## 2. Initialize SUMO and Data Collection

Now let's run a test simulation and collect some data.

In [11]:
!which sumo
!which sumo-gui

/opt/homebrew/bin/sumo
/opt/homebrew/bin/sumo-gui


In [10]:
# SUMO Installation Check

import os

def check_sumo_installation():
    """Verify SUMO installation and paths"""
    
    print("=== SUMO Installation Check ===")
    
    # Check Homebrew SUMO installation
    brew_sumo = '/opt/homebrew/opt/sumo'
    print(f"Checking Homebrew SUMO path ({brew_sumo}):", 
          "EXISTS" if os.path.exists(brew_sumo) else "NOT FOUND")
    
    # Check SUMO binaries
    sumo_path = os.popen('which sumo').read().strip()
    sumo_gui_path = os.popen('which sumo-gui').read().strip()
    print(f"SUMO binary: {sumo_path}")
    print(f"SUMO-GUI binary: {sumo_gui_path}")
    
    # Check SUMO tools
    tools_path = os.path.join(brew_sumo, 'share', 'sumo', 'tools')
    print(f"SUMO tools path ({tools_path}):",
          "EXISTS" if os.path.exists(tools_path) else "NOT FOUND")
    
    # Try to import traci
    try:
        import traci
        print("TraCI import: SUCCESS")
    except ImportError as e:
        print(f"TraCI import: FAILED - {e}")
    
    print("\nIf all paths exist and TraCI imports successfully, you're ready to run the simulation.")
    return brew_sumo, tools_path

# Run the check
sumo_home, tools_path = check_sumo_installation()

=== SUMO Installation Check ===
Checking Homebrew SUMO path (/opt/homebrew/opt/sumo): EXISTS
SUMO binary: /opt/homebrew/bin/sumo
SUMO-GUI binary: /opt/homebrew/bin/sumo-gui
SUMO tools path (/opt/homebrew/opt/sumo/share/sumo/tools): EXISTS
TraCI import: SUCCESS

If all paths exist and TraCI imports successfully, you're ready to run the simulation.


In [14]:
import os
import traci
import sys
import random
from pathlib import Path

def close_all_traci_connections():
    """Close any existing TraCI connections"""
    try:
        traci.close()
    except:
        pass
    
    # Additional cleanup for any remaining connections
    if hasattr(traci, '_connections'):
        for conn in traci._connections:
            try:
                conn.close()
            except:
                pass

# Close any existing connections first
close_all_traci_connections()

# For MacOS Homebrew SUMO installation
if 'SUMO_HOME' not in os.environ:
    brew_sumo = '/opt/homebrew/opt/sumo'
    if os.path.exists(brew_sumo):
        os.environ['SUMO_HOME'] = brew_sumo
        print(f"Set SUMO_HOME to {brew_sumo}")
    else:
        raise EnvironmentError("Could not find SUMO installation in Homebrew location")

# Add SUMO tools to Python path
tools = os.path.join(os.environ['SUMO_HOME'], 'share', 'sumo', 'tools')
if tools not in sys.path:
    sys.path.append(tools)

# Print current working directory and project structure
print(f"Current working directory: {os.getcwd()}")

# Get the absolute path to the project root (assuming we're in the notebooks directory)
project_root = Path(os.getcwd()).parent
print(f"Project root: {project_root}")

# Define the expected config file path
sumo_config_dir = project_root / 'data' / 'sumo_nets' / 'boston'
sumo_config_path = sumo_config_dir / 'morning_rush.sumocfg'

# Create directories if they don't exist
os.makedirs(sumo_config_dir, exist_ok=True)

# Check if config file exists
print(f"\nChecking configuration:")
print(f"Looking for config file at: {sumo_config_path}")
if not sumo_config_path.exists():
    print("Config file not found. Have you run the network setup script yet?")
    print("Let's create a basic test configuration to verify SUMO connection.")
    
    # Create a minimal test network and configuration
    test_net_path = sumo_config_dir / 'test.net.xml'
    test_config_path = sumo_config_dir / 'test.sumocfg'
    
    # Create a basic test network
    with open(test_net_path, 'w') as f:
        f.write("""<?xml version="1.0" encoding="UTF-8"?>
<net version="1.9" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/net_file.xsd">
    <location netOffset="0.00,0.00" convBoundary="0.00,0.00,100.00,0.00" origBoundary="-10000000000.00,-10000000000.00,10000000000.00,10000000000.00" projParameter="!"/>
    <edge id="1" from="1" to="2" priority="-1">
        <lane id="1_0" index="0" speed="13.89" length="100.00" shape="0.00,-1.60 100.00,-1.60"/>
    </edge>
    <junction id="1" type="dead_end" x="0.00" y="0.00" incLanes="" intLanes="" shape="0.00,0.00 0.00,-3.20"/>
    <junction id="2" type="dead_end" x="100.00" y="0.00" incLanes="1_0" intLanes="" shape="100.00,-3.20 100.00,0.00"/>
</net>
""")
    
    # Create a basic test configuration
    with open(test_config_path, 'w') as f:
        f.write("""<?xml version="1.0" encoding="UTF-8"?>
<configuration xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/sumoConfiguration.xsd">
    <input>
        <net-file value="test.net.xml"/>
    </input>
    <time>
        <begin value="0"/>
        <end value="100"/>
    </time>
</configuration>
""")
    
    sumo_config_path = test_config_path
    print(f"Created test configuration at: {test_config_path}")

try:
    # Start SUMO with a random port
    port = random.randint(8000, 9000)
    print(f"\nStarting SUMO:")
    print(f"Port: {port}")
    print(f"Config: {sumo_config_path}")
    
    sumo_cmd = [
        'sumo-gui',
        '-c', str(sumo_config_path),
        '--start',
        '--quit-on-end',
        '--error-log', str(sumo_config_dir / 'sumo_errors.log')
    ]
    
    print(f"Command: {' '.join(sumo_cmd)}")
    
    # Start SUMO with specific connection label
    traci.start(sumo_cmd, port=port, label='simulation_1')
    print("Successfully connected to SUMO!")
    
    # Initialize data collector
    from src.utils.data_collector import SUMODataCollector
    collector = SUMODataCollector()
    collector.setup_collection()

    # Run simulation for a few steps as a test
    print("\nRunning test simulation...")
    for step in range(10):  # Reduced steps for testing
        traci.simulationStep()
        collector.step_collection()
        print(f"Step {step}/10 completed")

except traci.exceptions.TraCIException as e:
    print(f"\nTraCI Exception: {e}")
    print("\nTroubleshooting steps:")
    print("1. Kill any existing SUMO processes:")
    print("   pkill -f sumo")
    print("2. Check if the port is in use:")
    print(f"   lsof -i :{port}")
    
except FileNotFoundError as e:
    print(f"\nFile not found: {e}")
    print("Please check the paths printed above and verify file locations.")
    
except Exception as e:
    print(f"\nUnexpected error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up
    close_all_traci_connections()
    print("\nClosed all SUMO connections.")

Current working directory: /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/notebooks
Project root: /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project

Checking configuration:
Looking for config file at: /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/data/sumo_nets/boston/morning_rush.sumocfg
Config file not found. Have you run the network setup script yet?
Let's create a basic test configuration to verify SUMO connection.
Created test configuration at: /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/data/sumo_nets/boston/test.sumocfg

Starting SUMO:
Port: 8799
Config: /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/data/sumo_nets/boston/test.sumocfg
Command: sumo-gui -c /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/data/sumo_nets/boston/test.sumocfg --start --quit-on-end --error-log /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/data/sumo_nets/boston/sumo_errors.log
 Retrying in 1 seconds
Successfully connected to SUMO!

Running test simulation...

Unexpect

Traceback (most recent call last):
  File "/var/folders/ns/_cr0srpd1v98_flffmvcgdhw0000gn/T/ipykernel_34033/179607008.py", line 124, in <module>
    collector.step_collection()
  File "/Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/src/utils/data_collector.py", line 121, in step_collection
    network_metrics = self.collect_network_metrics()
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/src/utils/data_collector.py", line 98, in collect_network_metrics
    mean_travel_time = 0
                  ^^^^^^^
AttributeError: 'VehicleDomain' object has no attribute 'getMeanSpeed'


In [19]:
import os
import sys
import traci
import random
from pathlib import Path
from src.utils.data_collector import SUMODataCollector

def create_test_files(sumo_config_dir):
    """Create minimal test network and route files"""
    # Create test network
    test_net_path = sumo_config_dir / 'test.net.xml'
    with open(test_net_path, 'w') as f:
        f.write("""<?xml version="1.0" encoding="UTF-8"?>
<net version="1.9">
    <location netOffset="0.00,0.00" convBoundary="0.00,0.00,200.00,0.00" origBoundary="-10000000000.00,-10000000000.00,10000000000.00,10000000000.00" projParameter="!"/>
    <edge id="1" from="1" to="2" priority="-1">
        <lane id="1_0" index="0" speed="13.89" length="200.00" shape="0.00,-1.60 200.00,-1.60"/>
    </edge>
    <junction id="1" type="dead_end" x="0.00" y="0.00" incLanes="" intLanes="" shape="0.00,0.00 0.00,-3.20"/>
    <junction id="2" type="dead_end" x="200.00" y="0.00" incLanes="1_0" intLanes="" shape="200.00,-3.20 200.00,0.00"/>
</net>
""")

    # Create route file
    test_route_path = sumo_config_dir / 'test.rou.xml'
    with open(test_route_path, 'w') as f:
        f.write("""<?xml version="1.0" encoding="UTF-8"?>
<routes>
    <vType id="car" accel="2.6" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="15"/>
    <route id="route_0" edges="1"/>
    <vehicle id="veh0" type="car" route="route_0" depart="0"/>
    <vehicle id="veh1" type="car" route="route_0" depart="2"/>
    <vehicle id="veh2" type="car" route="route_0" depart="4"/>
</routes>
""")

    # Create SUMO config
    test_config_path = sumo_config_dir / 'test.sumocfg'
    with open(test_config_path, 'w') as f:
        f.write("""<?xml version="1.0" encoding="UTF-8"?>
<configuration>
    <input>
        <net-file value="test.net.xml"/>
        <route-files value="test.rou.xml"/>
    </input>
    <time>
        <begin value="0"/>
        <end value="100"/>
    </time>
</configuration>
""")
    
    return test_config_path

def test_data_collection():
    """Test the minimal data collector"""
    # Setup paths
    project_root = Path(os.getcwd()).parent
    sumo_config_dir = project_root / 'data' / 'sumo_nets' / 'boston'
    
    # Create test files
    os.makedirs(sumo_config_dir, exist_ok=True)
    sumo_config_path = create_test_files(sumo_config_dir)
    
    print(f"Created test files in: {sumo_config_dir}")
    
    try:
        # Close any existing connections
        try:
            traci.close()
        except:
            pass
            
        # Start SUMO
        port = random.randint(8000, 9000)
        sumo_cmd = [
            'sumo-gui',
            '-c', str(sumo_config_path)
        ]
        
        print(f"\nStarting SUMO on port {port}...")
        traci.start(sumo_cmd, port=port)
        
        # Initialize collector
        collector = SUMODataCollector()
        collector.setup_collection()
        
        print("\nRunning simulation steps...")
        for step in range(10):
            traci.simulationStep()
            collector.step_collection()
            
            # Print metrics
            metrics = collector.collected_data['traffic_metrics'][-1]
            print(f"\nStep {step}:")
            print(f"- Vehicles: {metrics['vehicle_count']}")
            print(f"- Avg Speed: {metrics.get('average_speed', 0):.2f}")
            print(f"- Total Waiting: {metrics['total_waiting']:.2f}")
            
        # Export data
        output_dir = project_root / 'data' / 'processed' / 'test_results'
        collector.export_data(str(output_dir))
            
    except Exception as e:
        print(f"\nError during test: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        try:
            traci.close()
            print("\nClosed SUMO connection")
        except:
            pass

# Run the test
print("Current working directory:", os.getcwd())
print("Testing data collection...")
test_data_collection()

Current working directory: /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/notebooks
Testing data collection...
Created test files in: /Users/xfu/bu-courses-repo/cds-ds-340/ds340-project/data/sumo_nets/boston

Starting SUMO on port 8751...
 Retrying in 1 seconds


KeyboardInterrupt: 

## 3. Analyze Collected Data

Let's look at some basic metrics from our simulation.

In [ ]:
# Load the exported data
metrics_df = pd.read_csv('data/processed/simulation_results/traffic_metrics.csv')
intersection_df = pd.read_csv('data/processed/simulation_results/intersection_data.csv')
vehicle_df = pd.read_csv('data/processed/simulation_results/vehicle_data.csv')

# Plot some basic metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Mean Speed over time
axes[0,0].plot(metrics_df['timestamp'], metrics_df['mean_speed'])
axes[0,0].set_title('Mean Speed over Time')
axes[0,0].set_xlabel('Simulation Time (s)')
axes[0,0].set_ylabel('Speed (m/s)')

# Vehicle Count over time
axes[0,1].plot(metrics_df['timestamp'], metrics_df['vehicle_count'])
axes[0,1].set_title('Vehicle Count over Time')
axes[0,1].set_xlabel('Simulation Time (s)')
axes[0,1].set_ylabel('Number of Vehicles')

# Total Waiting Time over time
axes[1,0].plot(metrics_df['timestamp'], metrics_df['total_waiting_time'])
axes[1,0].set_title('Total Waiting Time over Time')
axes[1,0].set_xlabel('Simulation Time (s)')
axes[1,0].set_ylabel('Waiting Time (s)')

# CO2 Emissions over time
axes[1,1].plot(metrics_df['timestamp'], metrics_df['total_co2'])
axes[1,1].set_title('Total CO2 Emissions over Time')
axes[1,1].set_xlabel('Simulation Time (s)')
axes[1,1].set_ylabel('CO2 (mg/s)')

plt.tight_layout()
plt.show()

## 4. Analyze Intersection Performance

Let's look at the performance of different intersections.

In [ ]:
# Calculate average queue length per intersection
def calculate_avg_queue_length(row):
    """Calculate average queue length from the queue_lengths dictionary"""
    queue_dict = eval(row['queue_lengths'])  # Convert string representation to dict
    return sum(queue_dict.values()) / len(queue_dict)

intersection_df['avg_queue_length'] = intersection_df.apply(calculate_avg_queue_length, axis=1)

# Plot average queue length for each intersection
plt.figure(figsize=(15, 6))
sns.boxplot(data=intersection_df, x='intersection_id', y='avg_queue_length')
plt.xticks(rotation=45)
plt.title('Average Queue Length by Intersection')
plt.xlabel('Intersection ID')
plt.ylabel('Average Queue Length')
plt.tight_layout()
plt.show()

## 5. Save Reference Intersections

Let's identify and save the IDs of key intersections for our RL agents.

In [ ]:
# Get unique intersection IDs
intersection_ids = intersection_df['intersection_id'].unique()

# Calculate average queue length for each intersection
intersection_stats = intersection_df.groupby('intersection_id')['avg_queue_length'].mean().sort_values(ascending=False)

# Save top 10 busiest intersections
busy_intersections = intersection_stats.head(10)
print("Top 10 busiest intersections:")
print(busy_intersections)

# Save to file
busy_intersections.to_csv('data/processed/busy_intersections.csv')